In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score,mean_absolute_error as mae
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import re
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
def convert1(s):
  l = []
  s = list(s)
  for i in s:
    if 'followers' in i:
      i = i.replace(" followers","")
    else:
      i = i  
    if 'k' not in i:
      u = i.replace("," , "")
      u = int(u)
    elif 'k' in i:
      u = float(i[0:len(i) - 1]) * 1000
    else:
      u = i  
    l.append(u)
  return l 



def data_pre(text):
  text = str(text).lower()
  text = re.sub(r"https\S+","",text,flags=re.MULTILINE)

  text = re.sub(r'\@w+|\#',"",text)

  text = re.sub(r'[^\w\s]' , "", text)

  text_tokens = word_tokenize(text)

  filtered_text = [w for w in text_tokens if not w in stop_words]

  f = " ".join(filtered_text)

  y = []

  for i in list(f.split(" ")):
    y.append(stemmer.stem(i))

  return " ".join(y)  

In [5]:
df = pd.read_csv('/content/DA-AI-ML-interview-assignment-Data (1).csv')

In [6]:
df['num_ratings'] = convert1(df['num_ratings'])
df['num_reviews'] = convert1(df['num_reviews'])
df['num_followers'] = convert1(df['num_followers'])

df['num_ratings'] = df['num_ratings'] / 1000

In [7]:
df.drop(['Unnamed: 0' , 'genre' , 'num_ratings'] , axis = 1 , inplace = True)

In [8]:
import warnings
warnings.filterwarnings('ignore')


dff = df[['title', 'name', 'num_reviews','num_followers', 'synopsis']]
dff['tags'] = dff['title'] + " " + dff['name'] + " " + dff['synopsis']

df2 = dff[['num_reviews' , 'num_followers', 'tags']]

df2['tags'] = df2.tags.apply(data_pre)

In [9]:
X = df2

X = pd.concat([X , df['rating']] , axis = 1)

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_lg

In [11]:
import spacy
nlp = spacy.load("en_core_web_lg")

X['vector'] = X['tags'].apply(lambda tags: nlp(tags).vector)

xm = np.stack(X['vector'])

P = pd.DataFrame(xm)
R = pd.concat([X.drop(['tags' , 'vector'] , axis = 1) , P] , axis = 1) 

In [12]:
z = R

A_train2, A_test2, b_train2, b_test2 = train_test_split(z.drop(['rating'] , axis = 1),z['rating'], test_size=0.20, random_state=42 ,shuffle = True)

In [13]:
def build3_model(model):
  model.fit(A_train2 , b_train2)
  b_pred2 = model.predict(A_test2)
  
  print("r2_score  :" , r2_score(b_test2 , b_pred2))
  print("mean squared error  :" , mse(b_test2 , b_pred2)) 
  print("root mean squared error  :" , mse(b_test2 , b_pred2) ** (0.5))
  print("best parameters  :" , model.best_params_)
  print()
  print()

In [14]:
import warnings
warnings.simplefilter("ignore")


model1 = ElasticNet()
params = {
    'alpha':[1 , 0.5 , 0.05 , 0.7],
    'l1_ratio':[0.1 , 0.001 , 0.005 , 0.2 , 0.002]
}

grid = GridSearchCV(estimator = model1 , param_grid = params , scoring = 'r2' , n_jobs = -1 , cv = 5)

print(cross_val_score(grid , A_train2 , b_train2 , cv = 5 , scoring = 'r2').mean)


<built-in method mean of numpy.ndarray object at 0x7fba3c139270>


In [15]:
build3_model(grid)

r2_score  : 0.15906696125727904
mean squared error  : 0.05072226389893163
root mean squared error  : 0.22521603828087294
best parameters  : {'alpha': 0.5, 'l1_ratio': 0.001}


